In [1418]:
!pip install colorama graphviz ipywidgets

In [1419]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1420]:
# Class for Mental Illness
class MentalIllness:
    def __init__(self, symptoms=None, conclusion=None):
        self.symptoms = symptoms
        self.conclusion = conclusion

    def __str__(self):
        return 'Mental Illness[' + ', '.join([str(att) for att in self.__dict__.values() if att is not None]) + ']'

    def reason(self):
        return ' and '.join(['{}={}'.format(att, value) for att, value in self.__dict__.items() if value is not None])

# Node class for Tree
class Node:
    def __init__(self, parent=None, depth=0, accept_child=None, reject_child=None, condition=None, illness: MentalIllness = None):
        self.parent = parent
        self.depth = depth
        self.accept_child = accept_child
        self.reject_child = reject_child
        self.condition = condition
        self.illness = illness

    def __str__(self):
        accept_child_str = "Yes" if self.accept_child else "No"
        reject_child_str = "Yes" if self.reject_child else "No"

        illness_details = (
            f"Symptoms: {self.illness.symptoms}, Conclusion: {self.illness.conclusion}"
            if self.illness else "None"
        )

        return (
            f"Level: {self.depth}, "
            f"Condition: {self.condition}, "
            f"Illness: {illness_details}, "
            f"Has Accept Child: {accept_child_str} {self.accept_child.illness.conclusion if self.accept_child else None}, "
            f"Has Reject Child: {reject_child_str} {self.reject_child.illness.conclusion if self.reject_child else None}"
        )

# Tree class
class Tree:
    def __init__(self, root: Node = None):
        self.root = root

    def visualize(self, node=None):
        if node is None:
            node = self.root

        level = node.depth

        if node.illness:
            print("  " * level + f"Condition: {node.condition}, Symptoms: {node.illness.symptoms}, Conclusion: {node.illness.conclusion}")
        else:
            print("  " * level + f"Condition: {node.condition}, No associated illness")

        if node.reject_child:
            print("  " * (level + 1) + "Reject Child:")
            self.visualize(node.reject_child)
        if node.accept_child:
            print("  " * (level + 1) + "Accept Child:")
            self.visualize(node.accept_child)


In [1421]:
# RDR Graph Visualization

import os
from graphviz import Digraph

def parse_file_rules(file_path):
    with open(file_path, "r") as file:
        lines = [line.rstrip() for line in file if line.strip()]
    def extract_condition_conclusion(text):
        parts = text.split(':')
        condition = parts[0].strip()
        conclusion = parts[-1].strip() if len(parts) > 1 else None
        if "obj.condition ==" in condition:
            condition = condition.split("obj.condition ==")[1].strip()
        else:
            condition = condition.strip()
        if conclusion and "obj.conclusion ==" in conclusion:
            conclusion = conclusion.split("obj.conclusion ==")[1].strip()
        is_none = (condition == "None" or not condition) and (conclusion == "None" or not conclusion)
        display_text = f"Condition: {condition}, Conclusion: {conclusion}" if not is_none else "None"
        return {
            'condition': condition,
            'conclusion': conclusion,
            'is_none': is_none,
            'display_text': display_text,
            'original': text.strip()
        }
    first_line = lines[0]
    node_data = extract_condition_conclusion(first_line)
    tree = {
        "data": node_data,
        "children": [],
        "level": 0,
        "position": 0
    }
    stack = [(tree, len(first_line) - len(first_line.lstrip()))]
    current_level = 0
    for line in lines[1:]:
        indent = len(line) - len(line.lstrip())
        node_data = extract_condition_conclusion(line)
        if indent > stack[-1][1]:
            current_level += 1
        while stack and indent <= stack[-1][1]:
            stack.pop()
            current_level -= 1
        node = {
            "data": node_data,
            "children": [],
            "level": current_level,
            "position": len(stack[-1][0]["children"])
        }

        if stack:
            parent = stack[-1][0]
            parent["children"].append(node)

        stack.append((node, indent))
    return tree

def print_tree(node, prefix=""):
    if not node["data"]["is_none"]:
        print(f"\n{prefix}{node['data']['display_text']}")

    children = node.get("children", [])
    for i, child in enumerate(children):
        if i == len(children) - 1:
            next_prefix = prefix + "    └── "
        else:
            next_prefix = prefix + "    ├── "
        print_tree(child, next_prefix)

def build_graph(tree, graph=None, parent=None, node_id=0, is_right=True):
    if graph is None:
        graph = Digraph(comment="Decision Tree", format="pdf")
        graph.attr(rankdir="TB")
        graph.attr('node', shape='box')
        graph.attr(nodesep='1.0')
        graph.attr(ranksep='0.8')

    current_id = f"node{node_id}"

    if not tree["data"]["is_none"]:
        label = f'Condition: {tree["data"]["condition"]}\nConclusion: {tree["data"]["conclusion"]}'
        graph.node(current_id, label, style='rounded')
        if parent:
            graph.edge(parent, current_id)
    else:
        graph.node(current_id, "", style='invis')
        if parent:
            graph.edge(parent, current_id, style='invis')

    children = tree.get("children", [])
    for i, child in enumerate(children):
        child_id = node_id * 10 + i + 1
        next_is_right = (i % 2 == 0)
        build_graph(child, graph, current_id, child_id, next_is_right)

    return graph

def mainVisualize(file_path):
    output_dir = "/content/drive/MyDrive/TUGAS KULIAH/TUGAS-RPP-RDR"
    if not os.path.exists(output_dir):
        print(Style.BRIGHT + Fore.RED + f"\nOutput directory does not exist: {output_dir}" + Style.RESET_ALL)
        return

    if os.path.exists(file_path):
        tree = parse_file_rules(file_path)

        print("\nTree Structure in Terminal:")
        print("==========================")
        print_tree(tree)

        print("\nGenerating Visual Graph...")
        graph = build_graph(tree)
        graph.attr(layout='dot')
        graph.attr(ordering='out')

        output_path = os.path.join(output_dir, "final_rules")
        graph.render(output_path, format="pdf", view=True)
        print(f"\nDecision tree saved as PDF at: {output_path}.pdf")
    else:
        print(Style.BRIGHT + Fore.RED + "\nFile .rules not found. Please check the file path." + Style.RESET_ALL)

In [1422]:
# Read Rules and Build Tree
import os

def clean_value(value):
    value = value.strip()
    if value.startswith('[') and value.endswith(']'):
        value = value[1:-1].strip()
    value = value.replace("'", "").replace('"', "").strip()
    return value

def parse_rules(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    root = None
    node_stack = []
    is_right_child = []

    for i, line in enumerate(lines):
        line = line.replace("    ", "\t")

        if not line.strip():
            continue

        level = line.count('\t')
        line = line.lstrip('\t')

        if len(is_right_child) <= level:
            is_right_child.extend([False] * (level + 1 - len(is_right_child)))

        if level == 0:
            line = line.replace("True : ", "")

        # Parse the line content
        parts = line.split(" : ")
        if len(parts) < 3:
            print(f"Skipping invalid line: {line}")
            if i == len(lines) - 1:
                break
            continue

        # Extract condition, symptoms, and conclusion
        condition_part = parts[0].replace("obj.condition == ", "").strip()
        symptoms_part = parts[1].replace("obj.symptoms == ", "").strip()
        conclusion_part = parts[2].replace("obj.conclusion == ", "").strip()

        # Clean the values
        condition = clean_value(condition_part)
        symptoms = clean_value(symptoms_part)
        conclusion = clean_value(conclusion_part)

        # Handle None conclusion cases before printing
        if conclusion == "None":
            if not is_right_child[level]:
                is_right_child[level] = True
            if i == len(lines) - 1:
                break
            continue

        # Create a MentalIllness object and Node
        illness = MentalIllness(
            symptoms=symptoms.split(", ") if symptoms else None,
            conclusion=conclusion
        )

        current_node = Node(
            condition=condition.split(", ") if condition else None,
            illness=illness,
            depth=level
        )

        # Build parent-child relationships
        if level == 0:
            root = current_node
            node_stack = [current_node]
        else:
            while len(node_stack) > level:
                node_stack.pop()

            parent_node = node_stack[-1]

            if not is_right_child[level] and parent_node.reject_child is None:
                parent_node.reject_child = current_node
                is_right_child[level] = True
            elif is_right_child[level]:
                is_right_child[level] = False
                if parent_node.accept_child is None:
                    parent_node.accept_child = current_node

            node_stack.append(current_node)

    return Tree(root=root)

In [1423]:
# Get Inference/Conclusion based on Current Rules/Knowledges
from colorama import Fore, Style

def get_inference(tree, input_symptoms):
    current_node = tree.root
    last_true = tree.root
    last_active = tree.root

    while current_node:
        # print("\nNext")
        # print("current_node", current_node)
        # print("last_true", last_true)
        # print("last_active", last_active)

        if current_node.condition:
            if current_node == tree.root:
                condition_met = True
            else:
                condition_met = set(cond.lower() for cond in current_node.condition).issubset(input_symptoms)

            if condition_met:
                last_true = current_node
                last_active = current_node
                if current_node.accept_child:
                    current_node = current_node.accept_child
                else:
                    return last_true.illness.conclusion, last_active
            else:
                last_active = current_node
                if current_node.reject_child:
                    current_node = current_node.reject_child
                else:
                    return last_true.illness.conclusion, last_active
        else:
            if last_true.illness and last_true.illness.conclusion:
                return last_true.illness.conclusion, last_active
            break

    return "No matching conclusion found for the provided symptoms."

def on_button_click_with_inference(b):
    global last_active
    global user_symptoms

    user_symptoms = set(symptom.strip().lower() for symptom in symptoms_input.value.split(",") if symptom.strip())
    print(f"\nSymptoms entered: {user_symptoms}")

    if not mental_health_tree:
        print(Style.BRIGHT + Fore.RED + "\nMental health tree not initialized" + Style.RESET_ALL)

    elif not user_symptoms:
        print(Style.BRIGHT + Fore.RED + "\nNo symptoms entered. Please input symptoms" + Style.RESET_ALL)

    else:
        conclusion, last_active = get_inference(mental_health_tree, user_symptoms)
        print(Style.BRIGHT + Fore.GREEN + f"Inferred Conclusion: {conclusion}" + Style.RESET_ALL)

    # Display the question
    print(Style.BRIGHT + Fore.YELLOW + "\nDo you agree with this conclusion?:" + Style.RESET_ALL)

    accept_button = widgets.Button(description="Yes")
    reject_button = widgets.Button(description="No, I'll enter a new illness")
    accept_button.on_click(accept_inference)
    reject_button.on_click(reject_inference)

    button_box = HBox([accept_button, reject_button])
    display(button_box)

In [1424]:
# Expand Tree to Add Illness/Rule
def expand_tree(tree, input_symptoms, input_illness, last_active) :
    # print("\nEXPAND TREE")
    # print("last_active", last_active)
    # print("input_symptoms", input_symptoms)

    last_active_symptoms = set(symptom.strip().lower() for symptom in last_active.illness.symptoms if symptom.strip())
    if last_active.illness and last_active.illness.symptoms:
        condition_diff = set(input_symptoms) - last_active_symptoms
    else:
        condition_diff = set(input_symptoms)

    if last_active == tree.root:
        condition_met = True
    else:
        condition_met = set(cond.lower() for cond in last_active.condition).issubset(input_symptoms)

    # Create new node
    new_illness = MentalIllness(
        symptoms = list(input_symptoms) if input_symptoms else None,
        conclusion = input_illness if isinstance(input_illness, str) else next(iter(input_illness), None)
    )

    new_node = Node(
        condition=list(condition_diff) if condition_diff else None,
        illness=new_illness,
        depth=last_active.depth + 1
    )

    # Assign new node
    if condition_met :
        if last_active.accept_child :
            print("Condition met but Accept Child already exists")
        else :
            last_active.accept_child = new_node
    else :
        if last_active.reject_child :
            print("Condition not met but Reject Child already exists")
        else :
            last_active.reject_child = new_node

    open(file_path, "w").close()

    # Save the tree
    save_tree(mental_health_tree.root, file_path)
    print(f"\nTree saved to {file_path}")

    mainVisualize(file_path)

In [1425]:
# Save new Tree
def save_tree(node, file_path, level=0, is_first_call=True):
    mode = "w" if is_first_call else "a"

    with open(file_path, mode) as file:
        if is_first_call:
            file.write("True : ")

        indent = "    " * level

        # Format node properties
        if node is None or (node.condition is None and node.illness is None):
            file.write(f"{indent}obj.condition == None : obj.symptoms == None : obj.conclusion == None\n")
        else:
            condition = f"obj.condition == {node.condition}" if node.condition else "obj.condition == None"
            if node.illness:
                symptoms = f"obj.symptoms == {node.illness.symptoms}" if node.illness.symptoms else "obj.symptoms == None"
                conclusion = f"obj.conclusion == {node.illness.conclusion}" if node.illness.conclusion else "obj.conclusion == None"

            else:
                symptoms = "obj.symptoms == None"
                conclusion = "obj.conclusion == None"

            # Write the formatted line
            file.write(f"{indent}{condition} : {symptoms} : {conclusion}\n")

    # Recursively process children if they exist
    if node and (node.accept_child or node.reject_child):
        # Process reject_child
        if node.reject_child:
            save_tree(node.reject_child, file_path, level + 1, False)
        else:
            with open(file_path, "a") as f:
                f.write(f"{indent}    obj.condition == None : obj.symptoms == None : obj.conclusion == None\n")

        # Process accept_child
        if node.accept_child:
            save_tree(node.accept_child, file_path, level + 1, False)
        else:
            with open(file_path, "a") as f:
                f.write(f"{indent}    obj.condition == None : obj.symptoms == None : obj.conclusion == None\n")


In [1426]:
# Expert Accept/Reject Conclusion
from ipywidgets import widgets, HBox
from IPython.display import display

def accept_inference(b):
    print(Style.BRIGHT + Fore.YELLOW + "\nThank you for confirming the conclusion!" + Style.RESET_ALL)\

def reject_inference(b):
    print("\n")
    illness_input = widgets.Text(
        value='',
        placeholder='Enter illness name here...',
        description='Illness:',
        disabled=False,
        layout=widgets.Layout(width='600px')
    )

    submit_illness_button = widgets.Button(description="Submit Illness")

    def submit_illness(btn):
        new_illness = set(illness.strip().lower() for illness in illness_input.value.split(",") if illness.strip())
        if new_illness:
            print(f"\nNew illness entered: {new_illness}")
            expand_tree(mental_health_tree, user_symptoms, new_illness, last_active)
        else:
            print(Style.BRIGHT + Fore.RED + "\nNo illness entered. Please try again." + Style.RESET_ALL)

    submit_illness_button.on_click(submit_illness)
    display(illness_input, submit_illness_button)

In [1427]:
from colorama import Fore, Style

def main():
    global file_path
    file_path = "/content/drive/MyDrive/TUGAS KULIAH/TUGAS-RPP-RDR/mental-illness.rules.txt"

    global file_path_new
    file_path_new = "/content/drive/MyDrive/TUGAS KULIAH/TUGAS-RPP-RDR/new-mental-illness.rules.txt"

    global mental_health_tree
    if os.path.exists(file_path):
        mental_health_tree = parse_rules(file_path)
        print("Mental Health Tree Structure:")
        mental_health_tree.visualize()
    else:
        print(f"File not found: {file_path}")
        return

    # Input symptoms
    global symptoms_input
    symptoms_input = widgets.Text(
        value='',
        placeholder='Type symptoms here...',
        description='Symptoms:',
        disabled=False,
        layout=widgets.Layout(width='600px')
    )

    submit_button = widgets.Button(description="Submit")
    submit_button.on_click(on_button_click_with_inference)

    print(Style.BRIGHT + Fore.YELLOW + "\nEnter symptoms below:" + Style.RESET_ALL)
    display(symptoms_input, submit_button)

if __name__ == "__main__":
    main()

Mental Health Tree Structure:
Condition: ['None'], Symptoms: ['None'], Conclusion: Healty
  Accept Child:
  Condition: ['Memory Loss'], Symptoms: ['Memory Loss'], Conclusion: Narcolepsy
    Reject Child:
    Condition: ['Anxiety', 'Social Withdrawal'], Symptoms: ['Anxiety', 'Social Withdrawal'], Conclusion: Specific Phobia
      Reject Child:
      Condition: ['Physical Complaints'], Symptoms: ['Anxiety', 'Physical Complaints'], Conclusion: Restless Legs Syndrome
        Reject Child:
        Condition: ['impulsivity', 'sensory sensitivities', 'speech delay', 'mood swings'], Symptoms: ['speech delay', 'anxiety', 'sensory sensitivities', 'mood swings', 'impulsivity'], Conclusion: adhd (attention-deficit/hyperactivity disorder)
        Accept Child:
        Condition: ['speech delay', 'social withdrawal', 'sadness/depression'], Symptoms: ['speech delay', 'physical complaints', 'social withdrawal', 'sadness/depression'], Conclusion: intellectual development disorder
      Accept Child:
  

Text(value='', description='Symptoms:', layout=Layout(width='600px'), placeholder='Type symptoms here...')

Button(description='Submit', style=ButtonStyle())


Symptoms entered: {'social withdrawal', 'memory loss', 'continuous nightmares', 'anxiety', 'hallucinations/delusions', 'impulsivity', 'sadness/depression'}
Inferred Conclusion: depersonalization disorder

Do you agree with this conclusion?:


Text(value='', description='Illness:', layout=Layout(width='600px'), placeholder='Enter illness name here...')

Button(description='Submit Illness', style=ButtonStyle())


New illness entered: {'acute stress disorder'}

Tree saved to /content/drive/MyDrive/TUGAS KULIAH/TUGAS-RPP-RDR/mental-illness.rules.txt

Tree Structure in Terminal:

Condition: True, Conclusion: Healty

    └── Condition: ['Memory Loss'], Conclusion: Narcolepsy

    └──     ├── Condition: ['Anxiety', 'Social Withdrawal'], Conclusion: Specific Phobia

    └──     ├──     ├── Condition: ['Physical Complaints'], Conclusion: Restless Legs Syndrome

    └──     ├──     ├──     ├── Condition: ['impulsivity', 'sensory sensitivities', 'speech delay', 'mood swings'], Conclusion: adhd (attention-deficit/hyperactivity disorder)

    └──     ├──     ├──     └── Condition: ['speech delay', 'social withdrawal', 'sadness/depression'], Conclusion: intellectual development disorder

    └──     ├──     └── Condition: ['physical complaints'], Conclusion: illness anxiety disorder

    └──     ├──     └──     ├── Condition: ['speech delay'], Conclusion: communication disorder

    └──     ├──     └──   